<a href="https://colab.research.google.com/github/hira-14/Leet-code-coding-questions/blob/main/04_collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "numpy<2" --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
#!pip install scikit-surprise --quiet

import pandas as pd
import numpy as np
from surprise import Dataset, Reader, KNNBasic, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_PATH = '/content/drive/MyDrive/ml-1m/ml-1m'

In [ ]:
df = pd.read_csv(DATA_PATH + '/fe_data.csv')

/tmp/ipython-input-4-3076981435.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_PATH + '/fe_data.csv')


In [ ]:
print("Data loaded. Shape",df.shape)

Data loaded. Shape (1000209, 36)


In [ ]:
df.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zipcode,title,genres,...,Sci-Fi,Thriller,War,Western,release_year,user_total_ratings,movie_total_ratings,prev_ts,recency_days,session_within_7d
0,1,3186,4,2000-12-31 22:00:19,F,1,10,48067,"Girl, Interrupted (1999)",Drama,...,0,0,0,0,1999,53,431,NaN,0,1
1,1,1270,5,2000-12-31 22:00:55,F,1,10,48067,Back to the Future (1985),Comedy|Sci-Fi,...,1,0,0,0,1985,53,2583,2000-12-31 22:00:19,0,1
2,1,1721,4,2000-12-31 22:00:55,F,1,10,48067,Titanic (1997),Drama|Romance,...,0,0,0,0,1997,53,1546,2000-12-31 22:00:55,0,1
3,1,1022,5,2000-12-31 22:00:55,F,1,10,48067,Cinderella (1950),Animation|Children's|Musical,...,0,0,0,0,1950,53,577,2000-12-31 22:00:55,0,1
4,1,2340,3,2000-12-31 22:01:43,F,1,10,48067,Meet Joe Black (1998),Romance,...,0,0,0,0,1998,53,344,2000-12-31 22:00:55,0,1


In [ ]:
# Create ratings dataframe with required columns
ratings = df[['user_id', 'movie_id', 'rating']].copy()

# Create Surprise reader and dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings, reader)

# Split into train/test (80/20)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

print(f"Train size: {trainset.n_ratings}")
print(f"Test size: {len(testset)}")
print("Sample train entry:", trainset.ur[0][:2])  # First user's first 2 ratings

Train size: 800167
Test size: 200042
Sample train entry: [(0, 4.0), (907, 4.0)]


In [ ]:
# Memory-based CF (User-User KNN)
# Configure KNN parameters
knn_params = {
    'k': 40,
    'sim_options': {
        'name': 'cosine',
        'user_based': True  # User-user similarity
    },
    'verbose': False
}

# Train KNN model
knn = KNNBasic(**knn_params)
knn.fit(trainset)

# Test predictions
knn_preds = knn.test(testset)

# Calculate RMSE
rmse_knn = accuracy.rmse(knn_preds)
print(f"KNN RMSE: {rmse_knn:.4f}")

# Save model
with open(DATA_PATH + '/knn_model.pkl', 'wb') as f:
    pickle.dump(knn, f)
print("KNN model saved")

RMSE: 0.9789
KNN RMSE: 0.9789
KNN model saved


In [ ]:
# Memory-based CF - Item-Item KNN
# Configure KNN parameters for item-item
knn_item_params = {
    'k': 40,
    'sim_options': {
        'name': 'cosine',
        'user_based': False  # Item-item similarity
    },
    'verbose': False
}

# Train KNN model
knn_item = KNNBasic(**knn_item_params)
knn_item.fit(trainset)

# Test predictions
knn_item_preds = knn_item.test(testset)

# Calculate RMSE
rmse_knn_item = accuracy.rmse(knn_item_preds)
print(f"Item-Item KNN RMSE: {rmse_knn_item:.4f}")

# Save model
with open(DATA_PATH + '/knn_item_model.pkl', 'wb') as f:
    pickle.dump(knn_item, f)
print("Item-Item KNN model saved")

RMSE: 1.0010
Item-Item KNN RMSE: 1.0010
Item-Item KNN model saved


In [ ]:
# Cell 4: Model-based CF (SVD Matrix Factorization)
# Configure SVD parameters
svd = SVD(
    n_factors=50,
    n_epochs=20,
    lr_all=0.005,
    reg_all=0.02,
    verbose=False
)

# Train SVD model
svd.fit(trainset)

# Test predictions
svd_preds = svd.test(testset)
rmse_svd = accuracy.rmse(svd_preds)
print(f"SVD RMSE: {rmse_svd:.4f}")

# Save model
with open(DATA_PATH + '/svd_model.pkl', 'wb') as f:
    pickle.dump(svd, f)
print("SVD model saved")

RMSE: 0.8739
SVD RMSE: 0.8739
SVD model saved


In [ ]:
# Cell 6: Generate Top-N Recommendations for User 1 (Updated)
def get_cf_recs(algo, user_id, n=10):
    # Get all movie IDs
    all_mids = ratings['movie_id'].unique()

    # Get movies user has already rated
    seen = set(ratings[ratings['user_id'] == user_id]['movie_id'])

    # Predict ratings for unseen movies
    preds = []
    for mid in all_mids:
        if mid not in seen:
            pred = algo.predict(user_id, mid).est
            preds.append((mid, pred))

    # Get top-N recommendations
    top_n = sorted(preds, key=lambda x: x[1], reverse=True)[:n]

    # Convert to DataFrame
    recs_df = pd.DataFrame(top_n, columns=['movie_id', 'predicted_rating'])

    # Add movie titles
    movie_titles = df[['movie_id', 'title']].drop_duplicates()
    recs_df = recs_df.merge(movie_titles, on='movie_id')

    return recs_df

# Generate recommendations using all models
user_id = 1
user_knn_recs = get_cf_recs(knn, user_id)
item_knn_recs = get_cf_recs(knn_item, user_id)
svd_recs = get_cf_recs(svd, user_id)

# Save recommendations
user_knn_recs.to_csv(DATA_PATH + '/cf_recs_user1_knn_user.csv', index=False)
item_knn_recs.to_csv(DATA_PATH + '/cf_recs_user1_knn_item.csv', index=False)
svd_recs.to_csv(DATA_PATH + '/cf_recs_user1_svd.csv', index=False)
print("Recommendations saved for user 1")

# Print sample recommendations
print("\nTop 5 User-User KNN recommendations:")
print(user_knn_recs.head())

print("\nTop 5 Item-Item KNN recommendations:")
print(item_knn_recs.head())

print("\nTop 5 SVD recommendations:")
print(svd_recs.head())

Recommendations saved for user 1

Top 5 User-User KNN recommendations:
   movie_id  predicted_rating  \
0       787                 5   
1       578                 5   
2       853                 5   
3      1360                 5   
4      1787                 5   

                                               title  
0                 Gate of Heavenly Peace, The (1995)  
1                        Hour of the Pig, The (1993)  
2                                       Dingo (1992)  
3  Identification of a Woman (Identificazione di ...  
4  Paralyzing Fear: The Story of Polio in America...  

Top 5 Item-Item KNN recommendations:
   movie_id  predicted_rating                                        title
0      1039               5.0                   Synthetic Pleasures (1995)
1      3209               5.0                  Loves of Carmen, The (1948)
2      3216               5.0        Vampyros Lesbos (Las Vampiras) (1970)
3       989               5.0    Schlafes Bruder (Brother of S

In [ ]:
# Cell 7: Analysis and Summary (Updated)
# Compare model performance
print("Collaborative Filtering Results:")
print(f"User-User KNN RMSE: {rmse_knn:.4f}")
print(f"Item-Item KNN RMSE: {rmse_knn_item:.4f}")
print(f"SVD RMSE: {rmse_svd:.4f}")

# Recommendation diversity analysis
def recommendation_diversity(recs_df):
    titles = recs_df['title'].tolist()
    unique_words = len(set(" ".join(titles).split()))
    return unique_words / len(titles)  # Words per title

# Blockbuster concentration analysis
def blockbuster_concentration(recs_df, top_movies, n=100):
    top_titles = set(top_movies.index[:n])
    return sum(1 for title in recs_df['title'] if title in top_titles) / len(recs_df)

# Get top 100 most rated movies
top_movies = df['title'].value_counts().head(100)

print("\nRecommendation Diversity:")
print(f"User-User KNN: {recommendation_diversity(user_knn_recs):.2f} unique words per title")
print(f"Item-Item KNN: {recommendation_diversity(item_knn_recs):.2f} unique words per title")
print(f"SVD: {recommendation_diversity(svd_recs):.2f} unique words per title")

print("\nBlockbuster Concentration (Top 100 movies):")
print(f"User-User KNN: {blockbuster_concentration(user_knn_recs, top_movies):.1%}")
print(f"Item-Item KNN: {blockbuster_concentration(item_knn_recs, top_movies):.1%}")
print(f"SVD: {blockbuster_concentration(svd_recs, top_movies):.1%}")

# Save summary
with open(DATA_PATH + '/cf_summary.txt', 'w') as f:
    f.write("Collaborative Filtering Performance Summary\n")
    f.write("="*50 + "\n")
    f.write(f"User-User KNN RMSE: {rmse_knn:.4f}\n")
    f.write(f"Item-Item KNN RMSE: {rmse_knn_item:.4f}\n")
    f.write(f"SVD RMSE: {rmse_svd:.4f}\n\n")

    f.write("Recommendation Diversity (Unique words per title)\n")
    f.write("="*50 + "\n")
    f.write(f"User-User KNN: {recommendation_diversity(user_knn_recs):.2f}\n")
    f.write(f"Item-Item KNN: {recommendation_diversity(item_knn_recs):.2f}\n")
    f.write(f"SVD: {recommendation_diversity(svd_recs):.2f}\n\n")

    f.write("Blockbuster Concentration (% from Top 100 movies)\n")
    f.write("="*50 + "\n")
    f.write(f"User-User KNN: {blockbuster_concentration(user_knn_recs, top_movies):.1%}\n")
    f.write(f"Item-Item KNN: {blockbuster_concentration(item_knn_recs, top_movies):.1%}\n")
    f.write(f"SVD: {blockbuster_concentration(svd_recs, top_movies):.1%}\n")

print("\nSummary saved to cf_summary.txt")

Collaborative Filtering Results:
User-User KNN RMSE: 0.9789
Item-Item KNN RMSE: 1.0010
SVD RMSE: 0.8739

Recommendation Diversity:
User-User KNN: 4.10 unique words per title
Item-Item KNN: 3.80 unique words per title
SVD: 4.80 unique words per title

Blockbuster Concentration (Top 100 movies):
User-User KNN: 0.0%
Item-Item KNN: 0.0%
SVD: 30.0%

Summary saved to cf_summary.txt


# Collaborative Filtering: Methodology & Results

## **1. Methodology**

Collaborative Filtering (CF) leverages patterns in user–item interactions to generate personalized recommendations, based on the assumption that users who agreed in the past will agree in the future.

In this pipeline, we implemented **three CF models**:

- **User-User KNN**:  
  Recommends movies liked by users similar to the target user, using cosine similarity between user rating profiles.
- **Item-Item KNN**:  
  Recommends movies similar to those the user has already liked, using cosine similarity between item (movie) rating vectors.
- **Matrix Factorization (SVD)**:  
  Learns latent user and item features via Singular Value Decomposition, uncovering underlying factors driving user preferences.

**Key Steps:**
1. **Data Preparation**:  
   - Used the `MovieLens-1M` dataset, retaining only user, movie, and rating columns for modeling.
   - Split the data into an 80% training and 20% test set.
2. **Model Training & Evaluation**:  
   - Trained each model on the training set and evaluated on the test set using Root Mean Squared Error (**RMSE**).
   - Generated Top-N recommendations for a sample user and analyzed the diversity and "blockbuster" bias in the results.

---

## **2. Analysis of Results**

### **Model Performance (RMSE)**
| Model                 | RMSE    |
|-----------------------|---------|
| User-User CF          | 0.979   |
| Item-Item CF          | 1.001   |
| Matrix Factorization  | 0.874   |

- **Interpretation:**  
  - Lower RMSE indicates better accuracy in predicting user ratings.
  - **SVD (Matrix Factorization)** outperforms both KNN models, confirming that model-based approaches can capture more complex patterns in user–item interactions.

---

### **Recommendation Diversity**

| Model                 | Unique Words per Title |
|-----------------------|-----------------------|
| User-User CF          | 4.10                  |
| Item-Item CF          | 3.80                  |
| Matrix Factorization  | 4.80                  |

- **Interpretation:**  
  - **SVD** provides the most diverse set of recommendations.
  - Higher diversity suggests broader coverage of user interests, reducing the risk of "echo chambers" or repetitive suggestions.

---

### **Blockbuster Concentration**

| Model                 | % Blockbusters (Top 100) |
|-----------------------|-------------------------|
| User-User CF          | 0.0%                    |
| Item-Item CF          | 0.0%                    |
| Matrix Factorization  | 30.0%                   |

- **Interpretation:**  
  - **SVD** is more likely to recommend popular ("blockbuster") movies, possibly reflecting mainstream taste patterns learned during factorization.
  - KNN models, by contrast, produce more niche or personalized recommendations, avoiding over-reliance on the most-rated movies.

---

### **Sample Recommendations**

- The top recommendations for a sample user differ by algorithm:
  - **User-User KNN:**  
    Surface lesser-known titles shared by users with similar taste.
  - **Item-Item KNN:**  
    Suggests movies with rating profiles most similar to the user’s previously liked movies.
  - **SVD:**  
    Recommends a mix of blockbusters and relevant titles, reflecting latent preference matching.

---

## **3. Key Insights**

- **Matrix Factorization (SVD)** achieves the lowest RMSE, highest diversity, and highest blockbuster concentration, indicating a strong ability to generalize but a potential popularity bias.
- **KNN Models** (User-User, Item-Item) provide more "serendipitous" and niche recommendations, which may appeal to users with unique tastes or who wish to discover less mainstream content.
- **Model Choice Tradeoff:**  
  There is a natural trade-off between **accuracy** (RMSE), **diversity**, and **novelty** in recommender systems. SVD balances accuracy and discovery, but hybrid systems can further improve personalization and fairness.



